In [99]:
def IntegersModP(p):
    class IntegerModP():
        def __init__(self, n):
            if (n is None):
                sl
            self.n = n % p
            self.field = IntegerModP
        def __add__(self, other): return IntegerModP(self.n + other.n)
        def __sub__(self, other): return IntegerModP(self.n - other.n)
        def __mul__(self, other): return IntegerModP(self.n * other.n)
        def __truediv__(self, other): return self * other.inverse()
        def __div__(self, other): return self * other.inverse()
        def __neg__(self): return IntegerModP(-self.n)
        def __eq__(self, other): return isinstance(other, IntegerModP) and self.n == other.n
        def __abs__(self): return abs(self.n)
        def __str__(self): return '%d (mod %d)' % (self.n, self.p)
        def __repr__(self): return '%d (mod %d)' % (self.n, self.p)
        def __pow__(self,m): return IntegerModP(self.n**m)
        def inverse(self):
            return IntegerModP(self.n**(self.p-2))
    IntegerModP.p = p
    IntegerModP.__name__ = 'F_%d' % (p)
    return IntegerModP

def MyEllipticCurve(p,myA=None,myB=None,myJ=None):
    class EllipticCurve():
        # Assumes x,y given as either int or IntegerModP. So if it is not an integer, assumes it is IntegerModP with correct p
        #     this may cause a problem
        def __init__(self,x,y,isPInf):
            self.whichCurve=EllipticCurve
            if (type(x)==int):
                self.x=self.whichCurve.groundField(x)
            else:
                self.x=x
            if (type(y)==int):
                self.y=self.whichCurve.groundField(y)
            else:
                self.y=y
            self.isPInf=isPInf
        # double and add are assuming that the curve is given with A,B coefficients rather than Legendre form
        #   does not check this. Potential problem.
        def double(self):
            if (self.isPInf):
                return self
            myGF = self.whichCurve.groundField
            if (self.y==myGF(0)):
                return self.whichCurve(2894,19204,True)
            s = myGF(3)*self.x**2+myGF(self.whichCurve.myA)
            s = s/(myGF(2)*self.y)
            xR = s**2-myGF(2)*self.x
            return self.whichCurve(xR,-self.y-s*(xR-self.x),False)
        # does multiply by num using multiply and square trick. Add and double here.
        def multiplyByN(self,num):
            num=bin(num)[2:][::-1]
            lgth=len(num)
            current=self
            answer=self.whichCurve(2894,19204,True)
            for i in range(lgth):
                if (num[i]=='1'):
                    answer=answer.ecAdd(current)
                current=current.double()
            return answer
        def ecAdd(self,other):
            if (self.isPInf):
                return other
            elif (other.isPInf):
                return self
            elif (self.x==other.x):
                if (self.y==-other.y):
                    # put two junk numbers for x,y because it will be PInf anyway
                    return self.whichCurve(2894,19204,True)
                else:
                    return self.double()
            else:
                s = (self.y - other.y)/(self.x-other.x)
                xR = s**2 - self.x - other.x
                return self.whichCurve(xR,-self.y-s*(xR-self.x),False)
        def __str__(self):
            return '(%s,%s, %r)' % (str(self.x),str(self.y),self.isPInf)
        # If not given in Legendre form myJ will be none and output none. Otherwise do the Jacobi embedding
        #      into P^3 with 4 homogenous coordinates
        def jacobiEmbedding(self):
            if(self.whichCurve.myJ is None):
                return None
            else:
                if self.isPInf:
                    return (self.whichCurve.groundField(0),self.whichCurve.groundField(1),self.whichCurve.groundField(1),self.whichCurve.groundField(1))
                else:
                    x2 = (self.x)**2
                    x3 = self.whichCurve.groundField(2)*(self.x)
                    jGF = self.whichCurve.groundField(self.whichCurve.myJ)
                    return (-self.whichCurve.groundField(2)*(self.y),
                            x2-jGF,x2+x3*jGF+jGF,
                            x2+x3+jGF)
        # undoes the Jacobi embedding assuming 2 is invertible
        def undoJacobi(self,embedded):
            if (embedded==(self.whichCurve.groundField(0),self.whichCurve.groundField(1),self.whichCurve.groundField(1),self.whichCurve.groundField(1))):
                return self.whichCurve(2894,19204,True)
            else:
                myConst=self.whichCurve.groundField(2)
                jGF=self.whichCurve.groundField(self.whichCurve.myJ)
                y = embedded[0]/(-myConst)
                x = embedded[3]-embedded[1]-myConst*jGF
                x = x/myConst
                return self.whichCurve(x,y,False)
        # takes two points on the elliptic curve assumed in Legendre form and give the Jacobi embedding of the sum
        # if not in Legendre form will cause error when trying to make None-1
        def addJacobi(self,other):
            jacobiP1=self.jacobiEmbedding()
            jacobiP2=other.jacobiEmbedding()
            myConst=self.whichCurve.groundField(self.whichCurve.myJ-1)
            x3=jacobiP1[3]*jacobiP2[1]*jacobiP1[0]*jacobiP2[2]+jacobiP1[2]*jacobiP2[0]*jacobiP1[1]*jacobiP2[3]
            y3=jacobiP1[3]*jacobiP2[1]*jacobiP1[1]*jacobiP2[3]-jacobiP1[2]*jacobiP2[0]*jacobiP1[0]*jacobiP2[2]
            z3=jacobiP1[3]*jacobiP1[2]*jacobiP2[3]*jacobiP2[2]-myConst*jacobiP1[0]*jacobiP1[1]*jacobiP2[0]*jacobiP2[1]
            t3=(jacobiP1[3]*jacobiP2[1])**2+(jacobiP1[2]*jacobiP2[0])**2
            return (x3,y3,z3,t3)
        def doubleJacobi(self):
            jacobiP1=self.jacobiEmbedding()
            myConst=self.whichCurve.groundField(2)
            A=jacobiP1[3]*jacobiP1[1]
            B=jacobiP1[3]*jacobiP1[2]
            C=jacobiP1[2]*jacobiP1[1]
            D=jacobiP1[0]*A
            E=jacobiP1[2]*D
            x3=myConst*E
            y3=A**2-B**2+C**2
            z3=B**2-A**2+C**2
            t3=A**2+B**2-C**2
            return (x3,y3,z3,t3)
        #does multiply by num using multiply and square trick. Add and double here.
        def multiplyByNJacobi(self,num):
            num=bin(num)[2:][::-1]
            lgth=len(num)
            current=self
            answer=self.whichCurve(2894,19204,True)
            for i in range(lgth):
                if (num[i]=='1'):
                    answer=answer.addJacobi(current)
                    answer=self.undoJacobi(answer)
                current=current.doubleJacobi()
                current=self.undoJacobi(current)
            return answer
    EllipticCurve.groundField=IntegersModP(p)
    EllipticCurve.myA=myA
    EllipticCurve.myB=myB
    EllipticCurve.myJ=myJ
    EllipticCurve.__name__= 'myA=%s, myB=%s, myJ=%s over %s' % (str(myA),str(myB),str(myJ),IntegersModP(p).__name__)
    def calcDisc(myA,myB,myJ,groundField):
        if (myA is None or myB is None):
            return None
        else:
            return groundField(-16)*(groundField(4)*groundField(myA)**3+groundField(27)*groundField(myB)**2)
    EllipticCurve.discriminant=calcDisc(myA,myB,myJ,EllipticCurve.groundField)
    return EllipticCurve

In [124]:
myF19=IntegersModP(19)
print('The name of the field is '+myF19.__name__)
print('In this field 9*10 = '+str(myF19(9)*myF19(10)))
result=pow(myF19(9),2)
print('In this field 9^2='+str(result))
result=myF19(9)**2
print('In this field 9**2='+str(result))

The name of the field is F_19
In this field 9*10 = 14 (mod 19)
In this field 9^2=5 (mod 19)
In this field 9**2=5 (mod 19)


In [119]:
myCurve=MyEllipticCurve(19,myJ=4)
print(myCurve.__name__)
point1=myCurve(4,0,False)
result=point1.jacobiEmbedding()
print('(4,0) gets embedded as '+str(result))
result=point1.undoJacobi(result)
print('Should recover (4,0): '+str(result))
point1=myCurve(4,0,True)
result=point1.jacobiEmbedding()
print('PInf gets embedded as '+str(result))
point1=myCurve(4,0,False)
result=point1.addJacobi(point1)
print('(4,0)+(4,0) gets embedded as '+str(result))
result=point1.multiplyByNJacobi(11)
print('11*(4,0) by double and add in Jacobi coordinates ' + str(result))
result=point1.multiplyByNJacobi(21)
print('21*(4,0) by double and add in Jacobi coordinates '+str(result))

myA=None, myB=None, myJ=4 over F_19
(4,0) gets embedded as (0 (mod 19), 12 (mod 19), 14 (mod 19), 9 (mod 19))
Should recover (4,0): (4 (mod 19),0 (mod 19), False)
PInf gets embedded as (0 (mod 19), 1 (mod 19), 1 (mod 19), 1 (mod 19))
(4,0)+(4,0) gets embedded as (0 (mod 19), 17 (mod 19), 11 (mod 19), 17 (mod 19))
11*(4,0) by double and add in Jacobi coordinates (4 (mod 19),0 (mod 19), False)
21*(4,0) by double and add in Jacobi coordinates (15 (mod 19),0 (mod 19), False)


In [123]:
myCurve=MyEllipticCurve(11,myA=4,myB=16)
print('discriminant='+str(myCurve.discriminant))
point1=myCurve(0,4,False)
print('P='+str(point1))
point2=point1.ecAdd(point1)
print('2P=' + str(point2))
for i in range(7):
    point2=point2.ecAdd(point1)
    print('%dP=' % (i+3) + str(point2))
print('8*above should be PInf: ' + str(point2.multiplyByN(8)))

discriminant=9 (mod 11)
P=(0 (mod 11),4 (mod 11), False)
2P=(3 (mod 11),0 (mod 11), False)
3P=(0 (mod 11),7 (mod 11), False)
4P=(1 (mod 11),9 (mod 11), True)
5P=(0 (mod 11),4 (mod 11), False)
6P=(3 (mod 11),0 (mod 11), False)
7P=(0 (mod 11),7 (mod 11), False)
8P=(1 (mod 11),9 (mod 11), True)
9P=(0 (mod 11),4 (mod 11), False)
8*above should be PInf: (1 (mod 11),9 (mod 11), True)
